In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [3]:
df_train = pd.read_csv('Titanic_train.csv')

In [4]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df_train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [13]:
print(df_train[df_train.Fare > 500])

     PassengerId  Survived  Pclass                                Name  \
258          259         1       1                    Ward, Miss. Anna   
679          680         1       1  Cardeza, Mr. Thomas Drake Martinez   
737          738         1       1              Lesurer, Mr. Gustave J   

        Sex   Age  SibSp  Parch    Ticket      Fare        Cabin Embarked  
258  female  35.0      0      0  PC 17755  512.3292          NaN        C  
679    male  36.0      0      1  PC 17755  512.3292  B51 B53 B55        C  
737    male  35.0      0      0  PC 17755  512.3292         B101        C  


In [14]:
def outlier_z_score(ys, times):
    mean_ys = np.mean(ys)
    std_ys = np.std(ys)
    z = [(y - mean_ys)/std_ys for y in ys]
    return np.where(np.abs(z) > times)

In [15]:
out2 = outlier_z_score(df_train.Fare, 3)
print(df_train.loc[out2])

#這些人的pclass都是1，郵輪應該也會有豪華票，看起來是合理的

     PassengerId  Survived  Pclass  \
27            28         0       1   
88            89         1       1   
118          119         0       1   
258          259         1       1   
299          300         1       1   
311          312         1       1   
341          342         1       1   
377          378         0       1   
380          381         1       1   
438          439         0       1   
527          528         0       1   
557          558         0       1   
679          680         1       1   
689          690         1       1   
700          701         1       1   
716          717         1       1   
730          731         1       1   
737          738         1       1   
742          743         1       1   
779          780         1       1   

                                                  Name     Sex   Age  SibSp  \
27                      Fortune, Mr. Charles Alexander    male  19.0      3   
88                          Fortune, Miss. 

In [16]:
def outlier_iqr(ys, times):
    q1, q3 = np.nanpercentile(ys, [25, 75])
    iqr = q3 - q1
    lower_bound = q1-(iqr*times)
    upper_bound = q3+(iqr*times)
    return np.where((ys>upper_bound)|(ys<lower_bound))

In [17]:
#1.5iqr
out3 = outlier_iqr(df_train.Fare, 1.5)
print(df_train.loc[out3].Fare)
#有116筆，iqr加大

1       71.2833
27     263.0000
31     146.5208
34      82.1708
52      76.7292
         ...   
846     69.5500
849     89.1042
856    164.8667
863     69.5500
879     83.1583
Name: Fare, Length: 116, dtype: float64


In [18]:
out4 = outlier_iqr(df_train.Fare, 2.5)
print(df_train.loc[out4].Fare)
#還是有63筆，有點多，不參考這個指標

27     263.0000
31     146.5208
88     263.0000
118    247.5208
195    146.5208
         ...   
779    211.3375
802    120.0000
820     93.5000
849     89.1042
856    164.8667
Name: Fare, Length: 63, dtype: float64


In [ ]:
#問題二：應該算是合理的，用3倍標準差計算時，發現離群值的pclass都是1，512那筆可能是總統套房的概念，對高級郵輪來說不會很奇怪